In [6]:
%run -i helpers.py
%matplotlib inline

In [7]:
load_dotenv('./.env')

True

In [4]:
cols_expect = ['Environmental','Social','Governance']
esg_labels = ['Environmental','Social','Governance']

In [7]:
transcrip = pd.read_pickle('./data/motley-fool-data.pkl')
transcrip['dt'] = transcrip.date.apply(lambda x: parse_dt(x))
transcrip.drop(columns = ['date','exchange'],inplace=True)

snp = pd.read_csv('./data/constituents_csv.csv')
snp.Symbol = snp.Symbol.astype(str)
constituents_snp = [x for x in list(transcrip.ticker.unique()) if x in snp.Symbol.tolist()]
print(f'we have {len(constituents_snp)} cons from S&P 500')
transcrip_snp = transcrip[transcrip.ticker.isin(constituents_snp)]

we have 474 cons from S&P 500


In [8]:
dt_tic = transcrip_snp.groupby(['ticker'])['dt'].max().reset_index()
transcrip_snp_latest = transcrip_snp.merge(dt_tic, on = ['ticker','dt'], how = 'inner')


In [9]:
transcrip_snp_latest.sort_values(['ticker','dt'],inplace=True)
transcrip_snp_latest.drop_duplicates(inplace=True)
print(transcrip_snp_latest.shape)
transcrip_snp_latest.head()

(478, 4)


,q,ticker,transcript,dt
327,2022-Q3,A,Prepared Remarks:\nOperator\nGood afternoon. T...,2022-08-16 16:30:00
55,2022-Q2,AAL,"Prepared Remarks:\nOperator\nGood morning, and...",2022-07-21 08:30:00
436,2021-Q3,AAP,Prepared Remarks:\nOperator\nWelcome to the Ad...,2021-11-16 08:00:00
259,2023-Q1,AAPL,"Prepared Remarks:\nOperator\nGood day, everyon...",2023-02-02 17:00:00
199,2022-Q3,ABBV,"Prepared Remarks:\nOperator\nGood morning, and...",2022-10-28 09:00:00


In [37]:
def get_climate_risks(row):
    event_responses = get_responses_arctic_ai_risk(row.transcript)
    all_risks, len_risks = clean_response(event_responses)
    row['climate_risks'] = all_risks
    row['climate_risk_count'] = len_risks
    return(row)

In [42]:
sampled_tics = transcrip_snp_latest.sample(n=100,random_state=42)
transcrip_snp_latest_100 = transcrip_snp_latest[transcrip_snp_latest.ticker.isin(sampled_tics.ticker.unique())]

In [43]:
climate_risks_df = transcrip_snp_latest_100.apply(get_climate_risks, axis = 1)
climate_risks_df.drop(columns='transcript',inplace=True)
climate_risks_df.to_csv('./data/arctic_climate_risks_df_100.csv',index=False)

In [7]:
# climate_risks_df = pd.read_csv('./data/arctic_climate_risks_df_100.csv')

In [6]:
def calc_last_6m_returns_risk(row, mnths=6):
    tic = row.ticker
    current_date = datetime.date.today()
    six_months_ago = current_date - datetime.timedelta(days=mnths*30)

    stock_prices = pd.DataFrame(yf.Ticker(tic).history(start=six_months_ago.strftime("%Y-%m-%d"),  end=current_date.strftime("%Y-%m-%d"))).reset_index()
    stock_prices['ticker'] = tic
    stock_prices.Date = pd.to_datetime(stock_prices.Date,utc=True)  
    stock_prices = stock_prices.groupby(['ticker']).apply(calc_returns)
    if stock_prices.empty:
        return(row)    
    else:
        row['returns_6m'] = stock_prices.daily_return.sum() 
        row['vol_6m'] = stock_prices.daily_return.sum() 
        return()

climate_risks_df['returns_6m'] = climate_risks_df.ticker.apply(calc_last_3m_returns)

ERROR CDAY: No data found, symbol may be delisted
ERROR CERN: No timezone found, symbol may be delisted
ERROR CTXS: No timezone found, symbol may be delisted
ERROR NLSN: No timezone found, symbol may be delisted
ERROR PEAK: No data found, symbol may be delisted
ERROR SIVB: Period '1mo' is invalid, must be one of ['1d', '5d']
ERROR XLNX: No timezone found, symbol may be delisted


In [5]:
# climate_risks_df.to_csv('./data/complete_data_arctic_risk_returns_100.csv',index=False)
climate_risks_df = pd.read_csv('./data/complete_data_arctic_risk_returns_100.csv')
climate_risks_df[['climate_risk_count','returns_6m']].corr()

,climate_risk_count,returns_6m
climate_risk_count,1.000000,-0.117329
returns_6m,-0.117329,1.000000


In [15]:
climate_risks_df[climate_risks_df.climate_risks.notna()].to_csv('./data/climate_risks_statements_examples.csv',index=False)

: 

In [8]:
sp_weights = pd.read_html(requests.get('https://www.slickcharts.com/sp500',
                      headers={'User-agent': 'Mozilla/5.0'}).text)[0]

sp_weights.rename(columns={'Portfolio%':'sp500_weight','Symbol':'ticker'}, inplace=True)
sp_weights.sp500_weight = sp_weights.sp500_weight.apply(lambda x: str(x).replace('%', '')).astype(float)
sp_weights.to_csv('./data/sp500_weights.csv',index=False)
